# Download file from EXTM3U and Concat to MP4 using FFMPEG

In [ ]:
import requests
import os
import subprocess

In [ ]:
all_data = [
    {
        'url': 'URL of M3U -- get list of url (small video files)',
        'save_folder': r'FOLDER TARGET' + '\\',
        'mp4_name': 'v_ep2'
    }
]

ffmpeg = r'PATH OF ffmpeg.exe'

In [ ]:
for d in all_data:
    mp4_name = d['mp4_name']
    url = d['url']
    save_folder = d['save_folder'] + mp4_name

    # Make sure the folder exists
    os.makedirs(save_folder, exist_ok=True)

    res = requests.get(url)
    
    l_url = res.text.split('\n')
    l_url = list(filter(lambda x: 'https:' in x, l_url))
    print(mp4_name, 'Length file', len(l_url))

    list_mp4 = []

    for i, url in enumerate(l_url, start=1):
        filename = url.split("/")[-1]
        filepath = os.path.join(save_folder, filename)

        list_mp4.append('file ' + filepath)

        if os.path.exists(filepath):
            print(f"Skipping {filename} ({i}/{len(l_url)})", end="\r", flush=True)
            continue

        # Download and save
        response = requests.get(url)
        response.raise_for_status()

        with open(filepath, "wb") as f:
            f.write(response.content)

        # Progress output
        print(f"Downloaded {filename} ({i}/{len(l_url)})", end="\r", flush=True)

    # After loop, print a newline to avoid overwriting
    print(f"\nFinished downloading {mp4_name}, total {len(l_url)} files")

    text_filepath = os.path.join(save_folder, 'file.txt')
    content = '\n'.join(list_mp4).replace('\\', '\\\\').encode('utf-8')
    with open(text_filepath, "wb") as f:
        f.write(content)

    mp4_filepath = save_folder + '\\' + mp4_name + '.mp4'
    if os.path.exists(mp4_filepath):
        print('Skip', mp4_filepath)
        continue

    batch_filepath = os.path.join(save_folder, 'b.bat')
    batch_code = r'"' + ffmpeg + '" -f concat -safe 0 -i "' + text_filepath + '" -c copy "' + mp4_filepath + '"'
    with open(batch_filepath, "w") as f:
        f.write(batch_code)

    # Run via cmd explicitly
    result = subprocess.run(f'cmd /c "{batch_filepath}"', shell=True, capture_output=True, text=True)

    # Output
    print("Return code:", result.returncode)
    print("stdout:\n", result.stdout)
    print("stderr:\n", result.stderr)
